In [ ]:
from google.colab import drive
import datetime
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ta

  Created wheel for ta: filename=ta-0.7.0-cp37-none-any.whl size=28716 sha256=86ca907ca34a30985d37f994621caccf186f8174dd64f877076b9ed2a39e72aa
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta


In [ ]:
import ta as ta
import numpy as np

In [ ]:
pd.options.mode.chained_assignment = None 
dfx = pd.DataFrame( columns = ["STOCK","CANDLE","PERIOD","mult","BUYTARGETACHIEVED","SELLTARGETACHIEVED","PROFITABLETRADES","LOSSTRADES", "TOTALTRADES","PNL"])
                               
STOCKS = ['NIFTY','BANKNIFTY','RELIANCE','HDFC','HDFCBANK','TATAMOTORS','TATASTEEL','ASHOKLEY','AXISBANK','AUROPHARMA','BHARATFORG','BPCL','BIOCON','BHARTIARTL','BANKBARODA','CIPLA','CADILAHC','DIVISLAB','DRREDDY','ESCORTS','GLENMARK','HINDALCO','HCLTECH','INFY','ICICIBANK','LUPIN','LT','MARUTI','SUNTV','SUNPHARMA','SBIN','BAJFINANCE','BATAINDIA','MGL','MINDTREE','JUBLFOOD','INDUSINDBK']


STOCKS = ['NIFTY','BANKNIFTY']
#STOCKS = ['MOTHERSUMI','NATIONALUM','NMDC','NTPC','SAIL','SIEMENS','TATACHEM','TATAMOTORS','TATAPOWER','VEDL','ADANIENT','ADANIPORTS','BEL','BHARTIARTL','BHEL','BPCL','CUMMINSIND','GMRINFRA','GRASIM','L&TFH']



minutess = [5,10,15,30,60,90,120]
#minutess = [120]


for instrument in STOCKS:
  print(instrument)
  stock = f"{instrument}-I.NFO"
  listt = []
  
  for minutes in minutess:
    df = pd.read_csv(f"/content/drive/My Drive/reports/{instrument} [NSE FAO] 25MAR2021(I-{minutes}m).csv")
    df.columns = ['DATETIME',f'Open{minutes}Min',f'High{minutes}Min',f'Low{minutes}Min',f'Close{minutes}Min','Volume','Value','OI','DQ','Unnamed: 9']

    df['DATETIME'] = df.iloc[:,0]
    #pd.to_datetime(df['DATETIME'], format='%Y-%m-%d %H:%M:%S')
    pd.to_datetime(df['DATETIME'], format='%d/%m/%Y %H:%M:%S  %p')
    for period in np.arange(2,20):
      for multiplier in np.arange(2,20):  
      
        #df = pd.read_csv(f"/content/drive/My Drive/reports/alltimeframe{stock}FINALLhalf.csv")


        #df = df.set_index('DATETIME')
        
        

        dfvolget = df.loc[(df[f'High{minutes}Min'] > 0)]

        
        noofrows = (dfvolget.shape[0])
        period = min(noofrows,period)
        dfvolget['ATR'] = np.NaN
        df['ATR'] = np.NaN
        dfvolget['ATR'] = ta.volatility.average_true_range(high = dfvolget[f'High{minutes}Min'],low = dfvolget[f'Low{minutes}Min'],close = dfvolget[f'Close{minutes}Min'],window= int(period))
        df['ATR'] = dfvolget['ATR']

        dfHIGHLOWATR = df.loc[(df['ATR'] > 0)]
        dfHIGHLOWATR[f'average{minutes}Min'] = (dfHIGHLOWATR[f'High{minutes}Min'] + dfHIGHLOWATR[f'Low{minutes}Min'])/2

        dfHIGHLOWATR['UPPERATR'] = dfHIGHLOWATR[f'average{minutes}Min'] + multiplier*dfHIGHLOWATR['ATR']
        dfHIGHLOWATR['LOWERATR'] = dfHIGHLOWATR[f'average{minutes}Min'] - multiplier*dfHIGHLOWATR['ATR']
        df['UPPERATR'] = dfHIGHLOWATR['UPPERATR']
        df['LOWERATR'] = dfHIGHLOWATR['LOWERATR']

        df['UPPERATR'] = (df['UPPERATR'].ffill())
        df['LOWERATR'] = (df['LOWERATR'].ffill())
            
        df['UPPERATR'] = (df['UPPERATR'].shift(1))
        df['LOWERATR'] = (df['LOWERATR'].shift(1))
        df[f'Openfortrade{minutes}Min'] = df[f'Open{minutes}Min'].shift(-1)


        dfz = pd.DataFrame(columns= df.columns)

        buytoploss = []
        sellstoploss = []
        traded = 0
        squareoff = 0
        buytraded = 0
        selltraded = 0
        buytradedat = 0
        selltradedat = 0
        df['PNL'] = np.NaN
        df['TRADE'] = np.NaN
        #df['TRADED'] = np.NaN

        group = (df.loc[(df[f'Close{minutes}Min'] > 0)])
        
          
        group.loc[group[f'Close{minutes}Min'] > group['UPPERATR'], 'TRADE'] = 'BUY'
        group.loc[group[f'Close{minutes}Min'] < group['LOWERATR'], 'TRADE'] = 'SELL'
        group.dropna(subset = ["TRADE"], inplace=True)

        trade_signal = group['TRADE']
        ts_shifted = trade_signal.shift(1)
        group['TRADE'] = np.where(trade_signal != ts_shifted, trade_signal, np.NaN)

        group.dropna(subset = ["TRADE"], inplace=True)

      
        
        group.loc[group['TRADE'] == 'BUY', 'TRADED'] = group[f'Openfortrade{minutes}Min']
        group.loc[group['TRADE'] == 'SELL', 'TRADED'] = group[f'Openfortrade{minutes}Min']

        trade_signal = group['TRADE']
        traded_signal = group['TRADED']
        ts_shifted = trade_signal.shift(1)
        trade_shifted = traded_signal.shift(1)
        group['PNL'] = np.where(trade_signal == 'BUY',trade_shifted - traded_signal,traded_signal - trade_shifted)

        dfz = group






        XP = (dfz.loc[(dfz['TRADE'] == "SELL") & (dfz['PNL'] > 0)])
        buytargetachieved = XP.shape[0]
        XP = (dfz.loc[(dfz['TRADE'] == "BUY") & (dfz['PNL'] > 0)])
        selltargetachieved = XP.shape[0]
        XP = (dfz.loc[(dfz['PNL'] < 0)])
        losstrades = XP.shape[0]
        XP = (dfz.loc[(dfz['PNL'] > 0)])
        profittrades = XP.shape[0]
        XP = 0
        totaltrades = profittrades + losstrades

        #experiment
        total = dfz['PNL'].sum()
        #dfz.to_csv(f"/content/drive/My Drive/reports/ATRREP.csv")
        #experiment




        report = [stock, minutes,period,multiplier, buytargetachieved, selltargetachieved,profittrades, losstrades, totaltrades, total]
              
        TOTAL = [total,report]
        listt.append(TOTAL)
        dfz = pd.DataFrame()




  report = max(listt)[-1]
  

  to_append = report
  a_series = pd.Series(to_append, index=dfx.columns)
  dfx = dfx.append(a_series, ignore_index=True)

  #print(dfz)
  #print(dfz.to_csv(f"/content/drive/My Drive/reports/{stock}max{minutes}EXPERIMENTWITHOUTfromMAY.csv"))
  listt.clear()
  dfz = pd.DataFrame()
  

print(dfx)  
#print(dfx.to_csv(f"/content/drive/My Drive/reports/ATRREP.csv"))



NIFTY
BANKNIFTY
             STOCK CANDLE PERIOD  ... LOSSTRADES TOTALTRADES       PNL
0      NIFTY-I.NFO     10      4  ...         12          24   2567.15
1  BANKNIFTY-I.NFO     15      3  ...          7          17  12946.60

[2 rows x 10 columns]


In [ ]:
dfx

,STOCK,CANDLE,PERIOD,mult,BUYTARGETACHIEVED,SELLTARGETACHIEVED,PROFITABLETRADES,LOSSTRADES,TOTALTRADES,PNL
0,NIFTY-I.NFO,10,4,2,7,5,12,12,24,2567.15
1,BANKNIFTY-I.NFO,15,3,2,6,4,10,7,17,12946.60
